In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:
from skimage import feature
import numpy as np
import imutils
import cv2
import sys
import math
import glob
import os
import natsort
from pathlib import Path
import shutil
import time
import scipy.io

from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
import pandas as pd
from matplotlib import pyplot as plt
import glob

from numpy import unique
from numpy import hstack
from numpy import vstack
from numpy import where
from matplotlib import pyplot
from sklearn.datasets import make_blobs

import xlwt
from xlwt import Workbook
import openpyxl
# %%
NAMES = ['1_tatta', '2_natta', '4_kuditta_mettu', '5_kuditta_nattal',\
        '6_kuditta_tattal', '8_tei_tei_dhatta', \
   '9_katti_kartari','10_utsanga', '11_mandi',\
            '13_tirmana','14_sarika', '15_joining']
# '6_kuditta_tattal',
# NAMES = ['15_joining', '8_tei_tei_dhatta', \
#    '9_katti_kartari','10_utsanga', '11_mandi',\
#             '13_tirmana','14_sarika', ]

# NAMES = ['6_kuditta_tattal']
            # = '/content/drive/My Drive/Final Result V1/Final Result V1/Resized_Histogram_Binning_HOOF'
ALL_CSVFILE = '/content/drive/My Drive/Final Result V1/Final Result V1/Resized_Histogram_Binning_HOOF/'
i = 1
f = open(os.path.join(ALL_CSVFILE, 'Results.csv'), 'a')
print('nomenclature',
      ',',
      'accuracy_mf_kf',
      ',',
      'accuracy_mf',
      ',',
      'accuracy_kf',
      ',',
      'precision_mf',
      ',',
      'precision_kf',
      ',',
      'recall_mf',
      ',',
      'recall_kf',
      ',',
      'f1_mf',
      ',',
      'f1_kf',
      file=f)
f.close()

for NAME in NAMES:
    NAME_ = NAME + ''
    NAME2 = '_Histogram_Binning_HOOF'

    NAME3 = NAME_ + NAME2

    PATH = os.path.join(
        # '/content/drive/My Drive/Final Result V1/Final Result V1/Resized_Histogram_Binning_HOOF'
        '/content/drive/My Drive/Final Result V1/Final Result V1/Resized_Histogram_Binning_HOOF/',
        NAME3)
    PATH = PATH + '/'
    

    extracted_path = os.path.join(PATH ,  '*.mat')
    print(extracted_path)

    SAVE_FILE = NAME + '.csv'

    nomenclature = NAME

    X_init = np.empty((1, 9576))
    Y_init = np.empty((1, 1))

    for filename in glob.glob(extracted_path):
        print(i, '\t', filename)
        head, tail = os.path.split(filename)
        # print(tail)

        mat = scipy.io.loadmat(filename)

        X_data = mat['X_final']
        Y_data = mat['Y_final']

        # print(X_data.shape, Y_data.shape)

        ###     For all train
        X_init = np.vstack((X_init, X_data))
        Y_init = np.vstack((Y_init, Y_data))
#%%
##########delete first row and column
# print(X_init.shape)
# print(Y_init.shape)
    X_init = X_init[1:, :]
    Y_init = Y_init[1:, :]

    X = np.nan_to_num(X_init)
    Y = Y_init.ravel()

    print(X.shape)
    print(Y.shape)

    #### Split dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(
        X, Y, stratify=Y, test_size=0.2,
        random_state=3)  # 70% training and 30% test

    y_train_len = len(y_train)
    n_mf_train = np.count_nonzero(y_train)
    n_kf_train = y_train_len - n_mf_train
    percent_mf_train = 100 * (n_mf_train / y_train_len)
    percent_kf_train = 100 * (n_kf_train / y_train_len)

    print(percent_mf_train, percent_kf_train)
    #########################################################################
    #########################################################################
    #########################################################################
    kernel = 'linear'
    #Create a svm Classifier
    clf = svm.SVC(kernel=kernel,
                  class_weight={
                      1: percent_kf_train / 10,
                      0: percent_mf_train / 10
                  })  # Linear Kernel
    # clf = svm.SVC(kernel=kernel)  # Linear Kernel

    #Train the model using the training sets
    clf.fit(X_train, y_train)

    #Predict the response for test dataset
    y_pred = clf.predict(X_test)
    # Print the confusion matrix

    accuracy = (metrics.accuracy_score(y_test, y_pred)) * 100

    cm = metrics.confusion_matrix(y_test, y_pred)

    TN = cm[0][0]
    FN = cm[1][0]
    TP = cm[1][1]
    FP = cm[0][1]

    accuracy_mf_kf = accuracy
    accuracy_mf = (TP / (TP + FN)) * 100
    accuracy_kf = (TN / (TN + FP)) * 100

    precision_mf = (TP / (TP + FP)) * 100
    precision_kf = (TN / (TN + FN)) * 100

    recall_mf = (TP / (TP + FN)) * 100

    recall_mf = (TP / (TP + FN)) * 100
    recall_kf = (TN / (TN + FP)) * 100

    f1_mf = 2 * precision_mf * recall_mf / (precision_mf + recall_mf)
    f1_kf = 2 * precision_kf * recall_kf / (precision_kf + recall_kf)

    train_accuracy = (clf.score(X_train, y_train)) * 100
    test_accuracy = (clf.score(X_test, y_test)) * 100

    f = open(os.path.join(ALL_CSVFILE, 'Results.csv'), 'a')
    print(nomenclature,
          ',',
          "{:.2f}".format(accuracy_mf_kf),
          ',',
          "{:.2f}".format(accuracy_mf),
          ',',
          "{:.2f}".format(accuracy_kf),
          ',',
          "{:.2f}".format(precision_mf),
          ',',
          "{:.2f}".format(precision_kf),
          ',',
          "{:.2f}".format(recall_mf),
          ',',
          "{:.2f}".format(recall_kf),
          ',',
          "{:.2f}".format(f1_mf),
          ',',
          "{:.2f}".format(f1_kf),
          file=f)
    f.close()

    print(
        nomenclature,
        ',',
        "{:.2f}".format(accuracy_mf_kf),
        ',',
        "{:.2f}".format(accuracy_mf),
        ',',
        "{:.2f}".format(accuracy_kf),
        ',',
        "{:.2f}".format(precision_mf),
        ',',
        "{:.2f}".format(precision_kf),
        ',',
        "{:.2f}".format(recall_mf),
        ',',
        "{:.2f}".format(recall_kf),
        ',',
        "{:.2f}".format(f1_mf),
        ',',
        "{:.2f}".format(f1_kf),
    )

/content/drive/My Drive/Final Result V1/Final Result V1/Resized_Histogram_Binning_HOOF/6_kuditta_tattal_Histogram_Binning_HOOF/*.mat
1 	 /content/drive/My Drive/Final Result V1/Final Result V1/Resized_Histogram_Binning_HOOF/6_kuditta_tattal_Histogram_Binning_HOOF/6_Kuditta_tattal_1_Dancer_1.mat
1 	 /content/drive/My Drive/Final Result V1/Final Result V1/Resized_Histogram_Binning_HOOF/6_kuditta_tattal_Histogram_Binning_HOOF/6_Kuditta_tattal_1_Dancer_2.mat
1 	 /content/drive/My Drive/Final Result V1/Final Result V1/Resized_Histogram_Binning_HOOF/6_kuditta_tattal_Histogram_Binning_HOOF/6_Kuditta_tattal_1_Dancer_3.mat
1 	 /content/drive/My Drive/Final Result V1/Final Result V1/Resized_Histogram_Binning_HOOF/6_kuditta_tattal_Histogram_Binning_HOOF/6_Kuditta_tattal_2_Dancer_1.mat
1 	 /content/drive/My Drive/Final Result V1/Final Result V1/Resized_Histogram_Binning_HOOF/6_kuditta_tattal_Histogram_Binning_HOOF/6_Kuditta_tattal_2_Dancer_2.mat
1 	 /content/drive/My Drive/Final Result V1/Final Re